In [1]:
import txjupyter
reactor = txjupyter.start()
txjupyter.klein(reactor, 'tcp:port=8080')

In [4]:
from klein import route
@route('/')
def home(request):
    return 'Hello, world!'

In [5]:
@route('/greet/<user>')
def froop(request, user):
    return 'Hello {}!'.format(user)

In [6]:
import json
from twisted.internet import defer
import treq

@route('/ip')
@defer.inlineCallbacks
def getip(request):
    url = 'http://httpbin.org/ip'
    response = yield treq.get(url)
    content = yield response.content()
    value = json.loads(content)
    ip = value['origin']
    defer.returnValue('From {}'.format(ip))

In [7]:
@route('/slow')
@defer.inlineCallbacks
def slow(request):
    url = 'http://httpbin.org/delay/10'
    response = yield treq.get(url)
    content = yield response.content()
    value = json.loads(content)
    first_ip = value['origin']
    response = yield treq.get(url)
    content = yield response.content()
    value = json.loads(content)
    second_ip = value['origin']    
    defer.returnValue('These are the same:{}=={}'.format(first_ip, second_ip))

In [8]:
@route('/less-slow')
def less_slow(request):
    url = 'http://httpbin.org/delay/10'
    d = treq.get(url)
    d.addCallback(lambda response: response.content())
    d.addCallback(json.loads)
    d.addCallback(lambda dct: dct['origin'])
    d2 = treq.get(url)
    d2.addCallback(lambda response: response.content())
    d2.addCallback(json.loads)
    d2.addCallback(lambda dct: dct['origin'])
    both = defer.gatherResults([d, d2])
    def do_format(ips):
        first_ip, second_ip = ips
        return 'These are the same:{}=={}'.format(first_ip, second_ip)
    both.addCallback(do_format)
    return both

In [10]:
from twisted.internet import task
import operator

@route('/add/<int:a>/<int:b>')
def add(request, a, b):
    d = task.deferLater(reactor, 2, operator.add, a, b)
    d.addCallback(str)
    return d

In [11]:
def remote_add(x, y):
    url = 'http://localhost:8080/add'
    d = treq.get(url + '/{}/{}'.format(x, y))
    d.addCallback(lambda response: response.content())
    return d

@route('/add4/<int:a>/<int:b>/<int:c>/<int:d>')
def add4(request, a, b, c, d):
    d = defer.gatherResults([remote_add(a, b), 
                             remote_add(c, d)])
    @d.addCallback
    def final_add(results):
        x, y = results
        return remote_add(x, y)
    return d

@route('/slow-add4/<int:a>/<int:b>/<int:c>/<int:d>')
@defer.inlineCallbacks
def slow_add4(request, a, b, c, d):
    x = yield remote_add(a, b)
    y = yield remote_add(c, d)
    res = yield remote_add(x, y)
    return str(res)